In [1]:
import pandas as pd
import numpy as np
import pymysql
from ast import literal_eval
import warnings; warnings.simplefilter('ignore')

In [2]:
connection = pymysql.connect(
    host='localhost',
    user='root',
    password='',
    db='dv_db',
)

In [3]:
sql = "SELECT id, movieId, genres, title, vote_count, vote_average, year, popularity FROM movies"

with connection.cursor() as cursor:
    cursor.execute(sql)
    result = cursor.fetchall()

In [4]:
connection.close()

In [5]:
movies = pd.DataFrame(result, columns =['id', 'movieId', 'genres', 'title', 'vote_count', 'vote_average', 'year', 'popularity'])

In [6]:
movies['genres'] = movies['genres'].apply(lambda x: x.split(','))

In [7]:
movies['genres']

0                             [Animation, Comedy, Family]
1                            [Adventure, Fantasy, Family]
2                                       [Romance, Comedy]
3                                [Comedy, Drama, Romance]
4                                                [Comedy]
                              ...                        
9214                                              [Drama]
9215                                  [Thriller, Romance]
9216                 [Adventure, Drama, History, Romance]
9217    [Action, Adventure, Drama, Horror, Science Fic...
9218                                 [Documentary, Music]
Name: genres, Length: 9219, dtype: object

In [8]:
#Top 15

In [9]:
s = movies.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
gen_md = movies.drop('genres', axis=1).join(s)

In [10]:
gen_md

,id,movieId,title,vote_count,vote_average,year,popularity,genre
0,862,1,Toy Story,5415.0,7.7,1995,21.94690,Animation
0,862,1,Toy Story,5415.0,7.7,1995,21.94690,Comedy
0,862,1,Toy Story,5415.0,7.7,1995,21.94690,Family
1,8844,2,Jumanji,2413.0,6.9,1995,17.01550,Adventure
1,8844,2,Jumanji,2413.0,6.9,1995,17.01550,Fantasy
...,...,...,...,...,...,...,...,...
9217,315011,163056,Shin Godzilla,152.0,6.6,2016,9.28552,Drama
9217,315011,163056,Shin Godzilla,152.0,6.6,2016,9.28552,Horror
9217,315011,163056,Shin Godzilla,152.0,6.6,2016,9.28552,Science Fiction
9218,391698,163949,The Beatles: Eight Days a Week - The Touring Y...,92.0,7.6,2016,7.07830,Documentary


In [11]:
def build_chart(genre, percentile=0.75):
    df = gen_md[gen_md['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['id','title', 'year', 'vote_count', 'vote_average', 'popularity']]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    
    qualified['wr'] = qualified.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    
    return qualified

In [12]:
build_chart('Science Fiction').head(10)

,id,title,year,vote_count,vote_average,popularity,wr
7648,27205,Inception,2010,14076,8,29.1081,7.846178
8613,157336,Interstellar,2014,11187,8,32.2135,7.809657
232,11,Star Wars,1977,6779,8,42.1497,7.701534
970,1891,The Empire Strikes Back,1980,5999,8,19.4710,7.668181
979,185,A Clockwork Orange,1971,3433,8,17.1126,7.475284
7488,19995,Avatar,2009,12114,7,185.0710,6.898045
7969,24428,The Avengers,2012,12001,7,89.8876,6.897158
8712,118340,Guardians of the Galaxy,2014,10014,7,53.2916,6.878573
8864,76341,Mad Max: Fury Road,2015,9629,7,29.3618,6.874166
2079,603,The Matrix,1999,9080,7,33.3663,6.867300


In [13]:
gen_md.genre.unique()

array(['Animation', 'Comedy', 'Family', 'Adventure', 'Fantasy', 'Romance',
       'Drama', 'Action', 'Crime', 'Thriller', 'Horror', 'History',
       'Science Fiction', 'Mystery', 'War', 'Foreign', 'Music',
       'Documentary', '', 'Western', 'TV Movie'], dtype=object)